## Model I/O part
### Length Based Example Selector

In [32]:
from langchain.chat_models import ChatOllama
from langchain.prompts import PromptTemplate, ChatPromptTemplate
from langchain.schema import BaseOutputParser
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.prompts.example_selector import LengthBasedExampleSelector
from langchain.prompts.example_selector.base import BaseExampleSelector

In [33]:
ollama = ChatOllama(
    model = "mistral:latest",
    temperature=0.1,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()]
)

In [35]:
examples_country_capital = [
{
"question": "What do you know about France?",
"answer": """
Here is what I know:
Capital: Paris
Language: French
Food: Wine and Cheese
Currency: Euro
""",
},
{
"question": "What do you know about Italy?",
"answer": """
I know this:
Capital: Rome
Language: Italian
Food: Pizza and Pasta
Currency: Euro
""",
},
{
"question": "What do you know about Greece?",
"answer": """
I know this:
Capital: Athens
Language: Greek
Food: Souvlaki and Feta Cheese
Currency: Euro
""",
},
]

#same var
example_prompt_template = """
Human: {question}
Ai:{answer} 
"""

example_template = PromptTemplate.from_template(example_prompt_template)

In [38]:
#random selector

from typing import Any, Dict


class RandomExampleSelector(BaseExampleSelector):

    def __init__(self):
        self.examples = examples_country_capital

    def add_example(self, example):
        return super().add_example(example)

    def select_examples(self, input_variables):
        from random import choice
        return [choice(self.examples)]
    
    

In [39]:
#lenth limit

# example_selector_iMade = LengthBasedExampleSelector(
#     examples = examples_country_capital,
#     example_prompt=example_template,
#     max_length=2000,
# )

example_selector_iMade = RandomExampleSelector(
    examples = examples_country_capital,
)


prompt_country_example = FewShotPromptTemplate(
    example_prompt = example_template,
    example_selector=example_selector_iMade,
    suffix="Human: What do you know about {country}?",
    input_variables=["country"],
)

print(prompt_country_example.format(country="germany"))

TypeError: Can't instantiate abstract class RandomExampleSelector with abstract method add_example

In [33]:
# chain = prompt_country_example | doNotReplyElse | ollama
chain = prompt_country_example | ollama


chain.invoke({
    "country":"Germany"
})

I know this:
Capital: Berlin
Language: German
Food: Sausages and Beer
Currency: Euro

AIMessage(content='I know this:\nCapital: Berlin\nLanguage: German\nFood: Sausages and Beer\nCurrency: Euro')